# Data Cleaning in Python
The examples and code in this notebook are taken from the [Kaggle Data Cleaning Challenge](https://www.kaggle.com/getting-started/52652)

Detailed explanations for important code snippets are provided by Mervat Abuelkheir as part of the CSEN1095 Data Engineering Course.

The goal of this notebook is to show how to handle missing values and noisy data.

Pay attention to the <span style="color:red"> <b> paragraphs in bold red</b></span>; they ask you to do something and provide input!


## Inspecting the data

First thing we need to do is import a dataset.

<br>We will work in this exercise with the <a href="https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016#NFL%20Play%20by%20Play%202009-2017%20(v4).csv"> NFL dataset</a>. The NFL dataset is available in the repository's data folder, and is made available on Kaggle. It contains all the regular season plays from the 2009-2016 NFL seasons. The dataset has 100 columns. Each play contains information on: game situation, players involved, results, and advanced metrics such as expected point and win probability values.

### Important note

<b>The NFL dataset is very big (200+ MB) and is not made available in the `data` folder in this repository. Download the dataset from the link provided, store it in your own data or working directory, and proceed with running the code below, taking care to change your directory according to your own data storage location.</b>

<b>For large datasets, you will be provided with a link through which you can download the data. Another option is to read from the html link, provided the data is well-organized to help with proper parsing.</b>


In [3]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
nfl_data = pd.read_csv("data/NFL Play by Play 2009-2017 (v4).csv")

# set seed for reproducibility
np.random.seed(0) 


/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Look at a few rows of the nfl_data file. A handful of missing data!
nfl_data.sample(4)

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
244485,2014-10-26,2014102607,18,3,1.0,00:39,1,939.0,12.0,TB,...,1.240299,0.225647,0.774353,0.245582,0.754418,0.225647,0.019935,-0.018156,0.038091,2014
115340,2011-11-20,2011112000,22,4,1.0,06:47,7,407.0,44.0,OAK,...,NaN,0.056036,0.943964,0.042963,0.957037,0.943964,0.013073,NaN,NaN,2011
68357,2010-11-14,2010111401,8,2,NaN,00:23,1,1823.0,0.0,CLE,...,NaN,0.365307,0.634693,0.384697,0.615303,0.634693,-0.019390,NaN,NaN,2010
368377,2017-09-24,2017092405,24,4,1.0,08:48,9,528.0,8.0,CLE,...,1.075660,0.935995,0.064005,0.921231,0.078769,0.064005,0.014764,0.003866,0.010899,2017


## See how many missing data points we have

Let's see how many we have in each column. Let's start with the NFL dataset. 

In [5]:
# get the number of missing data points per column
missing_values_count_nfl = nfl_data.isnull().sum()

# look at the # of missing points in the first ten columns
print(missing_values_count_nfl)

Date                                0
GameID                              0
Drive                               0
qtr                                 0
down                            61154
time                              224
TimeUnder                           0
TimeSecs                          224
PlayTimeDiff                      444
SideofField                       528
yrdln                             840
yrdline100                        840
ydstogo                             0
ydsnet                              0
GoalToGo                          840
FirstDown                       28811
posteam                         24992
DefensiveTeam                   24992
desc                                2
PlayAttempted                       0
Yards.Gained                        0
sp                                  0
Touchdown                           0
ExPointResult                  397578
TwoPointConv                   407083
DefTwoPoint                    407664
Safety      

There are a lot of missing values it seems! Let's take a look at the percentage of the missing values in the dataset.

In [6]:
# How many total missing values do we have?
# shape returns the dimentionality of a dataframe (rows and columns), can you guess what product will do?
total_cells_nfl = np.product(nfl_data.shape) 
total_missing_nfl = missing_values_count_nfl.sum()

# percent of data that is missing
percentage_missign_values_nfl = (total_missing_nfl/total_cells_nfl) * 100
print(percentage_missign_values_nfl)

24.87214126835169


Almost a quarter of the cells in this dataset are empty! 

In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

## Examining the missing values

This is the point where you should develop some data intuition and look at the data to try and figure out why it is the way it is and how that will affect your analysis. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

**Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probably do want to keep as `NaN`. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, notice that the column TimesSec has a lot of missing values in it: 

In [7]:
# look at the # of missing points in the first ten columns
missing_values_count_nfl[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

The `TimeSecs` column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say which team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

**Tip: This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.**

You should look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Dropping the missing values

If you don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: DO NOT do this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [4]:
# remove all the rows that contain a missing value
nfl_data.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


Oops! That's removed all our data! <span style="color:red"><b>Why do you think this happened?</b></span>

We might have better luck removing all the columns that have at least one missing value instead.

In [5]:
# Remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1) # axis 1 indicates operation along columns
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,0,None,3,3,3,3,3,0.0,0.0,2009


Let's check how much data we lost.

In [6]:
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 102 

Columns with na's dropped: 41


We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

# Filling in missing values automatically

Another option is to try and fill in the missing values. Let's get a small sub-section of the NFL data so that it will print well and work with that to avoid having to work with the entire dataset and consume CPU time. You can work on the entire dataset at home!

In [7]:
# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [8]:
# replace all NA's with 0
subset_nfl_data.fillna(0) 

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


We could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [9]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
subset_nfl_data.fillna(method = 'bfill', axis=0).fillna(0) # here you may want to explore other methods like 'mean' if it makes sense

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


## Trying multivariate imputation with sklearn

sklearn has an `Imputer` and a `SimpleImputer` modules which perform imputation on one column at a time using the mean, median, or mode. 

sklearn also has a multivariate imputation module, `IterativeImputer`, that can help us impute values across multiple columns, but it is experimental and only available through developer release.

Let's see how an sklearn's imputation works for the NFL dataset.

In [10]:
from sklearn.preprocessing import Imputer
#from sklearn.impute import SimpleImputer # pretty much the same thing, you can try yourself
mean_imputer = Imputer(missing_values=np.nan, strategy='mean')
#mean_simpleimputer = SimpleImputer(missing_values=np.nan, strategy='mean') # Same use for sklearn SimpleImputer

# Again, let's get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()

# Check the presence of null values in the subset chosen
subset_nfl_data.isnull().sum()


/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


EPA             0
airEPA          3
yacEPA          3
Home_WP_pre     0
Away_WP_pre     0
Home_WP_post    0
Away_WP_post    0
Win_Prob        0
WPA             0
airWPA          3
yacWPA          3
Season          0
dtype: int64

In [11]:
# Now let's impute missing values in yacWPA column with the mean
mean_imputer.fit(subset_nfl_data[["yacWPA"]])
subset_nfl_data[["yacWPA"]]=mean_imputer.transform(subset_nfl_data[["yacWPA"]]).ravel()
#imputed_subset_nfl_data = pd.DataFrame(mean_simpleimputer.fit_transform(subset_nfl_data), columns=subset_nfl_data.columns) # fitting and transforming can be done in one step

# Let's check the presence null values again to see if imputation worked for the column
subset_nfl_data.isnull().sum()
# add you own counting of nulls for the dataframe imputed by SimpleImputer


EPA             0
airEPA          3
yacEPA          3
Home_WP_pre     0
Away_WP_pre     0
Home_WP_post    0
Away_WP_post    0
Win_Prob        0
WPA             0
airWPA          3
yacWPA          0
Season          0
dtype: int64

In [12]:
# Print the dataframe 
print(subset_nfl_data)
#print(imputed_subset_nfl_data) # if you want to try SimpleImputer code
# You may want to try the same code snippets above with the entire dataset and not just the subset

        EPA    airEPA    yacEPA  Home_WP_pre  Away_WP_pre  Home_WP_post  \
0  2.014474       NaN       NaN     0.485675     0.514325      0.546433   
1  0.077907 -1.068169  1.146076     0.546433     0.453567      0.551088   
2 -1.402760       NaN       NaN     0.551088     0.448912      0.510793   
3 -1.712583  3.318841 -5.031425     0.510793     0.489207      0.461217   
4  2.097796       NaN       NaN     0.461217     0.538783      0.558929   

   Away_WP_post  Win_Prob       WPA    airWPA    yacWPA  Season  
0      0.453567  0.485675  0.060758       NaN -0.059670    2009  
1      0.448912  0.546433  0.004655 -0.032244  0.036899    2009  
2      0.489207  0.551088 -0.040295       NaN -0.059670    2009  
3      0.538783  0.510793 -0.049576  0.106663 -0.156239    2009  
4      0.441071  0.461217  0.097712       NaN -0.059670    2009  


Now let's try the more advanced `IterativeImputer` in sklearn. It provides means to do multivariate imputation using a variety of ML models.

In [13]:
# This code snippet may not work, so do not worry about it, and just examine the similarities with the SimpleImputer
# To use this experimental feature, we need to explicitly ask for it:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge # Imputation via linear regression
from sklearn.neighbors import KNeighborsRegressor # Imputation via KNN
# Define imputer
reg_imputer = IterativeImputer(BayesianRidge, max_iter=5, random_state=0)
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
imputed_subset_nfl_data = pd.DataFrame(mean_simpleimputer.fit_transform(subset_nfl_data), columns=subset_nfl_data.columns)
print(imputed_subset_nfl_data)

ModuleNotFoundError: No module named 'sklearn.experimental'

## <span style="color:red"> Exercise 1: Missing value handling on the San Francisco Building Permits Dataset </span>

<span style="color:red"><b> The <a href="https://www.kaggle.com/aparnashastry/building-permit-applications-data#Building_Permits.csv"> Building permits dataset</a> pertains to all types of structural permits from Jan 1, 2013-Feb 25th 2018 in San Francisco. Data includes details on application/permit numbers, job addresses, supervisorial districts, and the current status of the applications. A data dictionary for the dataset is available in the data folder in this repository.</b></span>

<span style="color:red"> <b>The dataset is rather big and does not exist in the "data" folder in this repository. Download the dataset from the link provided and store it in your own data or working folder. Import the dataset in the next cell, and work your way through the steps in the subsequent cells.</b></span>

In [4]:
building_permits_data = pd.read_csv("data/Building_Permits.csv")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
building_permits_data.sample(10)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
40553,201403039652,8,otc alterations permit,03/03/2014,3732,008,400,NaN,Clementina,St,...,NaN,NaN,1.0,constr type 1,NaN,6.0,South of Market,94103.0,"(37.780460571778164, -122.40450626524974)",1334094491645
169731,201510159735,3,additions alterations or repairs,10/15/2015,2609,028,79,NaN,Buena Vista,Tr,...,5.0,wood frame (5),5.0,wood frame (5),NaN,8.0,Castro/Upper Market,94117.0,"(37.76757916496494, -122.43793170417105)",1399356139170
19180,M409787,8,otc alterations permit,07/22/2013,4624,031,178,NaN,West Point,Rd,...,NaN,NaN,NaN,NaN,NaN,10.0,Bayview Hunters Point,94124.0,"(37.73524725436046, -122.38063828309745)",1311685491725
68047,201411191888,8,otc alterations permit,11/19/2014,0039,109,294,NaN,Francisco,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,North Beach,94133.0,"(37.805257822817126, -122.40998545760392)",1362881288870
64238,M527228,8,otc alterations permit,10/14/2014,1251,002,707,NaN,Cole,St,...,NaN,NaN,NaN,NaN,NaN,5.0,Haight Ashbury,94117.0,"(37.76836885973765, -122.45074431487859)",135886493776
27703,M434527,8,otc alterations permit,10/11/2013,0348,015,277,NaN,Golden Gate,Av,...,NaN,NaN,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.781433365757565, -122.41487518938789)",1320626301305
109281,201512175360,8,otc alterations permit,12/17/2015,0061,013,40,NaN,Whiting,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,North Beach,94133.0,"(37.80389212201969, -122.40804427557475)",140688357279
15075,201306109020,8,otc alterations permit,06/10/2013,0262,020,100,NaN,Pine,St,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Financial District/South Beach,94111.0,"(37.792635570892635, -122.39899511726274)",1307390420381
59124,201408264765,8,otc alterations permit,08/26/2014,0321,016,650,NaN,Ellis,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,6.0,Tenderloin,94109.0,"(37.78461816669083, -122.41700445197698)",1353527233881
175167,201702240086,8,otc alterations permit,02/24/2017,6533,028,451,NaN,Fair Oaks,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,8.0,Mission,94110.0,"(37.74948705332759, -122.42343069698013)",1454274206536


<span style="color:red"> <b>Find out what percent of the building permits dataset is missing. </b></span>
<span style="color:red"> <b><br>Write down your code in the following cell. </b></span>

In [7]:
missing_values_count_buildings_permits = building_permits_data.isnull().sum()
total_cells_buildings_permit = np.product(building_permits_data.shape) 
total_missing_buildings_permit = missing_values_count_buildings_permits.sum()
percentage_missing_values_buildings_permit = (total_missing_buildings_permit/total_cells_buildings_permit) * 100
print(percentage_missing_values_buildings_permit)

26.26002315058403


<span style="color:red"><b>Write a code to check the values of the `Street Number Suffix` and `Zipcode` from the building permits datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?</b></span>

In [8]:
print(building_permits_data["Zipcode"])
print(building_permits_data["Street Number Suffix"])

print(missing_values_count_buildings_permits["Zipcode"]) #not recorded
print(missing_values_count_buildings_permits["Street Number Suffix"]) #doesn't exist

0         94102.0
1         94102.0
2         94109.0
3         94109.0
4         94102.0
5         94107.0
6         94122.0
7         94124.0
8         94117.0
9         94117.0
10        94114.0
11        94102.0
12        94114.0
13        94131.0
14        94115.0
15        94108.0
16        94104.0
17        94114.0
18        94123.0
19        94114.0
20        94117.0
21        94110.0
22        94133.0
23        94123.0
24        94123.0
25        94122.0
26        94121.0
27        94117.0
28        94122.0
29        94102.0
           ...   
198870    94109.0
198871    94117.0
198872    94114.0
198873    94110.0
198874    94131.0
198875    94102.0
198876    94117.0
198877    94112.0
198878    94110.0
198879    94112.0
198880    94122.0
198881    94133.0
198882    94112.0
198883    94102.0
198884    94103.0
198885    94116.0
198886        NaN
198887        NaN
198888        NaN
198889        NaN
198890        NaN
198891        NaN
198892        NaN
198893        NaN
198894    

<span style="color:red"><b>See if you can drop the missing values from the building permits dataset and still keep some data, or drop the columns with missing values and see if you have columns still.</b></span>

In [10]:
# Drop missing values
building_permits_data.dropna()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID


In [16]:
# Drop columns with missing values
columns_with_na_dropped = building_permits_data.dropna(axis=1) 
columns_with_na_dropped.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992


<span style="color:red"><b>Try replacing all the `NaN`'s in the building permits data with the one that comes directly after it and then replacing any remaining `NaN`'s with 0.</b></span>

In [17]:
building_permits_data_fill_NA = building_permits_data.fillna(method = 'bfill', axis=0).fillna(0) 

<span style="color:red"><b>Create a new dataset at the ending with all missing values handled and store as a CSV file.</b></span>

In [18]:
building_permits_data_fill_NA.to_csv(r'/Users/raneem/Downloads/CSEN1095-Data-Engineering-master 2/data/building_permits_no_missing.csv')


## <span style="color:red"> Exercise 2: Missing value handling on the  Pima Indians Diabetes Dataset</span>

<span style="color:red"><b> The <a href= "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"> Pima Indians Diabetes Dataset</a> involves predicting the onset of diabetes within 5 years in Pima Indians given medical details.</b></span>

<span style="color:red">There are 768 observations with 8 input variables and 1 output variable. The variable names are as follows:</span>

<br><span style="color:red">0. Number of times pregnant.</span>
<br><span style="color:red">1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.</span>
<br><span style="color:red">2. Diastolic blood pressure (mm Hg).</span>
<br><span style="color:red">3. Triceps skinfold thickness (mm).</span>
<br><span style="color:red">4. 2-Hour serum insulin (mu U/ml).</span>
<br><span style="color:red">5. Body mass index (weight in kg/(height in m)^2).</span>
<br><span style="color:red">6. Diabetes pedigree function.</span>
<br><span style="color:red">7. Age (years).
<br><span style="color:red">8. Class variable (0 or 1).</span>

<span style="color:red"> <b>This is a medical dataset, so an important thing we will do with this dataset is that we will see if the statistical summaries can reveal something about the missing values.</b></span>

<span style="color:red"> <b>The dataset is available in the `data` folder in this repository. Import the dataset in the next cell, and work your way through the steps in the subsequent cells.</b></span>

In [21]:
pima_indians_diabetes_data = pd.read_csv("data/pima-indians-diabetes.data.csv")
pima_indians_diabetes_data.sample(10)

,6,148,72,35,0,33.6,0.627,50,1
507,2,84,50,23,76,30.4,0.968,21,0
107,3,83,58,31,18,34.3,0.336,25,0
398,3,193,70,31,0,34.9,0.241,25,1
144,0,102,75,23,0,0.0,0.572,21,0
710,5,126,78,27,22,29.6,0.439,40,0
69,2,100,66,20,90,32.9,0.867,28,1
132,8,84,74,31,0,38.3,0.457,39,0
270,2,108,62,32,56,25.2,0.128,21,0
252,0,86,68,32,0,35.8,0.238,25,0
343,8,95,72,0,0,36.8,0.485,57,0


<span style="color:red"> <b>Use the describe method used for pandas data frames to obtain a statistucal summary of the dataset.</b></span>

In [22]:
pima_indians_diabetes_data.describe()

,6,148,72,35,0,33.6,0.627,50,1
count,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000
mean,3.842243,120.859192,69.101695,20.517601,79.903520,31.990482,0.471674,33.219035,0.348110
std,3.370877,31.978468,19.368155,15.954059,115.283105,7.889091,0.331497,11.752296,0.476682
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243500,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,32.000000,32.000000,0.371000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,127.500000,36.600000,0.625000,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


<span style="color:red"><b>There are columns that have a minimum value of zero (0). On some columns, a value of zero does not make sense and indicates an invalid or missing value. For example, the `BMI` for a person cannot be 0. See if there are other columns/attributes whose value of 0 does not make sense. An understanding of the medical columns may be needed.</b></span>

### Columns where 0 doesn't make sense : 
 - Plasma glucose concentration a 2 hours in an oral glucose tolerance test
 - Diastolic blood pressure (mm Hg)
 - BMI
 - Triceps skinfold thickness (mm)
 - Age

<span style="color:red"><b>Now you can proceed to check the missing values and remove/impute as in exercise 1. Create a new dataset at the ending with all missing values handled and store as a CSV file.</b></span>

In [33]:
from sklearn.impute import SimpleImputer
mean_imputer = SimpleImputer(missing_values=0, strategy='mean')

mean_imputer.fit(pima_indians_diabetes_data[["72"]])
pima_indians_diabetes_data[["72"]]=mean_imputer.transform(pima_indians_diabetes_data[["72"]]).astype(int).ravel()

mean_imputer.fit(pima_indians_diabetes_data[["35"]])
pima_indians_diabetes_data[["35"]]=mean_imputer.transform(pima_indians_diabetes_data[["35"]]).astype(int).ravel()

mean_imputer.fit(pima_indians_diabetes_data[["33.6"]])
pima_indians_diabetes_data[["33.6"]]=mean_imputer.transform(pima_indians_diabetes_data[["33.6"]]).round(1).ravel()

mean_imputer.fit(pima_indians_diabetes_data[["148"]])
pima_indians_diabetes_data[["148"]]=mean_imputer.transform(pima_indians_diabetes_data[["148"]]).astype(int).ravel()

median_imputer = SimpleImputer(missing_values=0, strategy='median')

median_imputer.fit(pima_indians_diabetes_data[["50"]])
pima_indians_diabetes_data[["50"]]=median_imputer.transform(pima_indians_diabetes_data[["50"]]).astype(int).ravel()
pima_indians_diabetes_data.sample(10)


,6,148,72,35,0,33.6,0.627,50,1
640,4,128,70,29,0,34.3,0.303,24,0
78,2,112,66,22,0,25.0,0.307,24,0
326,10,179,70,29,0,35.1,0.200,37,0
377,4,156,75,29,0,48.3,0.238,32,1
255,3,111,56,39,0,30.1,0.557,30,0
391,1,131,64,14,415,23.7,0.389,21,0
684,2,129,74,26,205,33.2,0.591,25,0
3,0,137,40,35,168,43.1,2.288,33,1
516,7,125,86,29,0,37.6,0.304,51,0
591,3,132,80,29,0,34.4,0.402,44,1


In [34]:
pima_indians_diabetes_data.to_csv(r'/Users/raneem/Downloads/CSEN1095-Data-Engineering-master 2/data/pima_indians_diabetes_no_missing_data.csv')